# SHO Fitting

This notebook completes the SHO fitting using the conventional algorithms in Pycroscopy. 

## Import Packages

In [ ]:
# # For the notebook to work you must have m3_learning installed
# # pip install m3_learning
%load_ext autoreload
%autoreload 2

from m3util.viz.printing import printer
from m3util.viz.style import set_style
from m3util.ml.rand import set_seeds
from m3util.util.IO import download_and_unzip
from belearn.dataset.dataset import BE_Dataset
from belearn.viz.viz import Viz

import numpy as np

# from m3_learning.be.dataset import BE_Dataset
printing = printer(basepath = './Figures/')

set_style("printing")
set_seeds(seed=42)

## Loading data for SHO fitting


In [ ]:
# Download the data file from Zenodo
url = 'https://zenodo.org/record/7774788/files/PZT_2080_raw_data.h5?download=1'

# Specify the filename and the path to save the file
# filename = 'data_raw.h5'
filename = '1_data_raw.h5'
save_path = './Data'

# download the file
download_and_unzip(filename, url, save_path)

In [ ]:
data_path = save_path + '/' + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

## Visualize Raw Data

### Raw Cantilever Response


In [ ]:
# insatiate the visualization object
image_scalebar = [2000, 500, "nm", "br"]

BE_viz = Viz(dataset, printing, verbose=True, 
             SHO_ranges = [(0,1.5e-4), (1.31e6, 1.33e6), (-300, 300), (-np.pi, np.pi)], 
             image_scalebar = image_scalebar)

In [ ]:
prediction = {"resampled": False,
              "label": "Raw"}

BE_viz.raw_data_comparison(prediction, filename="Figure_2_1_raw_cantilever_response")

**Figure 2.1** Raw cantilever response. a) magnitude spectrum and Phase, b) real and imaginary parts of the complex spectrum.


### Band-Excitation Experiments

In [ ]:
BE_viz.raw_be(dataset, filename="Figure_2_2_raw_be_experiment")

**Figure 2.2** Band-Excitation Experiments. a) raw drive amplitude applied to the tip, b) energy of the waveform in the frequency domain, c) DC voltage applied to switch the sample, inset shows a zoomed in view of the switching waveform where measurements are made in the on and off state. d) raw cantilever response magnitude and phase, e) real and imaginary parts of the complex spectrum.

## SHO Fitting
**Note**: this code takes around 15 minutes to execute

If you downloaded the SHO fit data, or ran notebook 1 you do not need to fit the data again. 

You can skip to the next section.

If you would like to run this code block please change the Fit_SHO flag to `True`

In [ ]:
Fit_SHO = False

if Fit_SHO: 

    # # computes the SHO fit for the data in the file
    dataset.SHO_Fitter(force = True)
    
    # instantiate the dataset object
    # good to reinstantiate the dataset object after fitting
    dataset = BE_Dataset(data_path)

### Views the Distribution of the Least Squares Fitting (LSQF) results

It is good to view the distributions and standardize the phase shift. The phase shift is rotated around the unit circle such that the peaks are at $-\pi/2$ and $\pi/2$

In [ ]:
dataset.LSQF_phase_shift = 0

BE_viz.SHO_hist(dataset.SHO_fit_results(),
                      filename="Figure_2_3_Original_LSQF_Histograms")


**Figure 2.3** Distribution of the Least Squares Fitting (LSQF) results of the unmodified data. a) Distribution of the amplitude, b) Distribution of the resonance frequency, c) distribution of the quality factor, d) distribution of the resonance frequency.

In [ ]:
dataset.LSQF_phase_shift = np.pi/2

BE_viz.SHO_hist(dataset.SHO_fit_results(),
                      filename="Figure_4_Phase_Shifted_LSQF_Histograms")

**Figure 2.4** Distribution of the Least Squares Fitting (LSQF) results of the normalized data. a) Distribution of the amplitude, b) distribution of the resonance frequency, c) distribution of the quality factor, d) distribution of the resonance frequency.

### Plots the SHO Fit Results

In [ ]:
BE_viz.dataset.measurement_state = 'on'

BE_viz.SHO_loops(filename="Figure_2_5_Single_Pixel_Loops")

**Figure 2_5.** Example switching loops obtained for the amplitude, resonance frequency, quality factor, and phase.

## SHO Switching Movies

In [ ]:
BE_viz.SHO_fit_movie_images(noise = 0, 
                            scalebar_= True, 
                            basepath = "Movies/SHO_LSQF_",  
                            filename="SHO_LSQF",
                            phase_shift = [np.pi/2])